<a href="https://colab.research.google.com/github/mslarson94/NMA2022_ECoG/blob/experimental/projects/ECoG/TA_single_trial_data_anaylsis_nback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading of Miller ECoG data of the memory nback task

includes some visualizations

In [1]:
# @title Data retrieval
import os, requests

fname = 'memory_nback.npz'
url = "https://osf.io/xfc7e/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [2]:
# @title Install packages (`nilearn`, `nimare`, `duecredit`), import `matplotlib` and set defaults
# install packages to visualize brains and electrode locations
!pip install nilearn --quiet
!pip install nimare --quiet
!pip install duecredit --quiet

from matplotlib import rcParams
from matplotlib import pyplot as plt

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] = 15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

     |████████████████████████████████| 9.6 MB 5.6 MB/s 
     |████████████████████████████████| 13.3 MB 8.7 MB/s 
     |████████████████████████████████| 3.2 MB 47.7 MB/s 
     |████████████████████████████████| 86 kB 3.8 MB/s 
     |████████████████████████████████| 179 kB 39.0 MB/s 


In [3]:
# @title Data loading
import numpy as np

alldat = np.load(fname, allow_pickle=True)['dat']

# # Select just one of the recordings here. This is subject 1, block 1.
# dat = alldat[1][1]

# print(dat.keys())

---
# Dataset info

This is one of multiple ECoG datasets from Miller 2019, recorded in clinical settings with a variety of tasks. Raw data here:

https://exhibits.stanford.edu/data/catalog/zk881ps0522

`dat` contain 3 sessions from 3 subjects, and was not yet used in a publication, so it's fresh data for you to analyze! The following references using other datasets may be useful for the memory n-back task: 

- Brouwer, Anne-Marie, et al. "Estimating workload using EEG spectral power and ERPs in the n-back task." Journal of Neural Engineering 9.4 (2012): 045008. doi: [10.1088/1741-2560/9/4/045008](https://doi.org/10.1088/1741-2560/9/4/045008)

- Grissmann, Sebastian, et al. "Electroencephalography based analysis of working memory load and affective valence in an n-back task with emotional stimuli." Frontiers in Human Neuroscience 11 (2017): 616. doi: [10.3389/fnhum.2017.00616](https://doi.org/10.3389/fnhum.2017.00616)

Each subject's data is divided into 5 sessions: fixation, exp1, exp2, exp3, fixation, which are consecutive blocks in the data structure (i.e. `alldat[0][0]`, `alldat[0][1]`, ..., `alldat[0][4]` for subject 0). Exp1 is a "0-back" memory experiment, where the subjects have to identify the picture of one of the houses (stimid = 10), which they have memorized at the beginning of the stimulus block. Exp2 is a 1-back memory experiment, where the participants have to respond to images of repeated houses. Finally, exp3 is 2-back where participants respond to pictures of houses that are repeated after another intervening random picture. This task was hard for these participants and most of them did not really respond at all, but the data can still be used to ask questions about overall shifts of neural activity in a hard task when the subjects feel overwhelmed by the memory requirements.

Note that for the third/last subject, no response data was collected, so only the first two subjects can be analyzed for their correct / incorrect trials.   

Sample rate is always 1000Hz, and the ECoG data has been notch-filtered at 60, 120, 180, 240 and 250Hz, followed by z-scoring across the entire recording and conversion to float16 to minimize size. 

Variables for each block within each subject: 
* `dat['V']`: continuous voltage data (time by channels)
* `dat['expinfo']`: experiment type for this block
* `dat['srate']`: sampling rate for voltage data, always 1000Hz
* `dat['t_on']`: time of stimulus onset in data samples
* `dat['t_off']`: time of stimulus offset, usually 600 samples after `t_on`
* `dat['stim_id`]: identity of house stimulus from 1 to 40. Stimulus 10 was the target in the 0-back task.  
* `dat['target']`: 0 or 1, indicates if this stimulus requires a response
* `dat['response']`: 0 or 1, indicates if the subject actually made a response
* `dat['rt']`: reaction time for trials with a response in voltage data samples (1000Hz). 
* `dat['locs']`: 3D locations of the electrodes on the cortical surface

In [ ]:
dat['locs'][27]

NameError: ignored

In [4]:

def get_voltage_patient_expirement_data(data, patient, experiment,freq_range):
    # compute spectral power above 50Hz and low-pass below 10Hz
  # power is always positive, so we normalize it by its average
  from scipy import signal
  dat=data[patient][experiment]
  V = dat['V'].astype('float32') # always convert the voltage data to float32!

  # high-pass filter above 50 Hz
  b, a = signal.butter(3, [freq_range[1]], btype='high', fs=1000)
  V = signal.filtfilt(b, a, V, 0)

  b, a = signal.butter(3, [freq_range[0]], btype='low', fs=1000)
  V = signal.filtfilt(b, a, V, 0)

  # compute smooth envelope of this signal = approx power
  V = np.abs(V)**2
  b, a = signal.butter(3, [freq_range[0]], btype='low', fs=1000)
  V = signal.filtfilt(b, a, V, 0)

  # normalize each channel so its mean power is 1
  V = V/V.mean(0)
  return V, dat




#should I get voltage 

In [ ]:
np.diff(dat['t_on'])[0]-(dat['t_off']-dat['t_on'])[0]

1600

In [81]:
store=[]
other_store=[]
BA_list=[6,7,8,9,10,32,40,45,46,47]
BA_chan_list=[None] * len(BA_list)
Lobe_chan_list=[None] *2*len(BA_list)
k=0
kk=0
for j in BA_list:
  print('Brodmann Area '+str(j))
  for i in range(63):
    if dat['Brodmann_Area'][i]=='Brodmann area '+str(j):
      store.append(i)
  BA_chan_list[k]=store
  for idx in ['Frontal Lobe', 'Parietal Lobe']:
    for jj in BA_chan_list[k]:
      if dat['lobe'][jj]==idx:
        other_store.append(jj)
    Lobe_chan_list[kk]=other_store
    other_store=[]
    kk+=1 

  store=[]
  other_store=[]
  k+=1

Frontal=Lobe_chan_list[:len(BA_list)]
Parietal=Lobe_chan_list[len(BA_list):]
print('Frontal Lobe List is ',Frontal)
print('Parietal Lobe List is',Parietal)

Brodmann Area 6
Brodmann Area 7
Brodmann Area 8
Brodmann Area 9
Brodmann Area 10
Brodmann Area 32
Brodmann Area 40
Brodmann Area 45
Brodmann Area 46
Brodmann Area 47
Frontal Lobe List is  [[20, 28, 36, 37, 44, 45, 51, 52, 53, 59, 60, 61], [], [], [], [43, 48, 49, 50, 56, 57, 58], [], [35, 40, 41, 42], [], [], []]
Parietal Lobe List is [[], [], [], [22, 31, 39], [16, 17, 18, 26], [], [24, 25, 32, 33, 34], [], [0, 1, 8, 9, 10], []]


In [84]:


BA_area_idx=0 
for chan in Frontal[BA_area_idx]:
  #chan=29
  subject_number=0
  experiment_number=1
  f_range=[4,8]
  V, dat=get_voltage_patient_expirement_data(alldat,subject_number,experiment_number,f_range)
  print(dat['lobe'][chan])
  print(dat['gyrus'][chan])
  print(dat['hemisphere'][chan])
  print(dat['Brodmann_Area'][chan])


  # divide into trials and average
  nt, nchan = V.shape
  nstim = len(dat['t_on'])

  # use a timerange from 500ms before to 1700ms after the stimulus onset
  trange = np.arange(-500, 1700)
  ts = dat['t_on'][:, np.newaxis] + trange
  V_epochs = np.reshape(V[ts, :], (nstim, 2200, nchan))

  V_onrp = (V_epochs[dat['target']==1])
  V_ofrp = (V_epochs[dat['target']==0])
  V_resp = (V_epochs[dat['response'] == 1])
  V_base = (V_epochs[dat['response'] == 0])


  plt.plot(V[:,chan])
  plt.vlines(dat['t_on'],10, 13,colors='y')
  plt.vlines(dat['t_on'][np.argwhere(dat['target']==1)],15,20,colors='r')
  plt.vlines(dat['t_on'][np.argwhere(dat['response']==1)],25,30,colors='g')
  plt.title(['Channel', chan, 'Subject,', subject_number,'Experiment', experiment_number, 'freq_range=', f_range])

  plt.figure(figsize=(25, 25))
  for j in range(100):
    ax = plt.subplot(10, 10, j+1)

    plt.plot(trange, V_epochs[j,: ,chan], lw=2)
    #plt.plot(trange, V_ofrp[j,:,chan], lw=2)
    if np.any(np.argwhere(dat['target']==1)==j):
      plt.plot(-300,7,'r',marker='*')
    if np.any(np.argwhere(dat['response']==1)==j):
      plt.plot(-300,9,'g',marker='D')
      plt.legend([dat['rt'][j]], loc='upper right',  title_fontsize='xx-small') #legend has response times in it

    plt.title('trial%d'%j)
    plt.yticks([])
    plt.ylim([0,10])
  plt.show()

KeyboardInterrupt: ignored

In [ ]:
# this electrode has a more transient response
plt.subplot(1, 3, 1)
plt.imshow(V_epochs[dat['response']==1, :,chan].astype('float32'),
           aspect='auto')
plt.colorbar()
plt.show()

In [7]:
print(dat['lobe'][chan])
print(dat['gyrus'][chan])
print(dat['hemisphere'][chan])
print(dat['Brodmann_Area'][chan])


Frontal Lobe
Precentral Gyrus
Right Cerebrum
Brodmann area 4


In [76]:
store=[]
other_store=[]
BA_list=[6,7,8,9,10,32,40,45,46,47]
BA_chan_list=[None] * len(BA_list)
Lobe_chan_list=[None] *2*len(BA_list)
k=0
kk=0
for j in BA_list:
  print('Brodmann Area '+str(j))
  for i in range(63):
    if dat['Brodmann_Area'][i]=='Brodmann area '+str(j):
      store.append(i)
  BA_chan_list[k]=store
  for idx in ['Frontal Lobe', 'Parietal Lobe']:
    for jj in BA_chan_list[k]:
      if dat['lobe'][jj]==idx:
        other_store.append(jj)
    Lobe_chan_list[kk]=other_store
    other_store=[]
    kk+=1 

  store=[]
  other_store=[]
  k+=1


Frontal=Lobe_chan_list[:len(BA_list)]
Parietal=Lobe_chan_list[len(BA_list):]

Brodmann Area 6
Brodmann Area 7
Brodmann Area 8
Brodmann Area 9
Brodmann Area 10
Brodmann Area 32
Brodmann Area 40
Brodmann Area 45
Brodmann Area 46
Brodmann Area 47


[]

In [72]:
len(Lobe_chan_list)

20

In [61]:
dat['lobe'][1]==Frontal Lobe

SyntaxError: ignored

In [63]:
idx

'Parietal Lobe'

In [51]:
print(BA_chan_list)

[[20, 28, 36, 37, 44, 45, 51, 52, 53, 59, 60, 61], [], [43, 48, 49, 50, 56, 57, 58], [35, 40, 41, 42], [], [], [22, 31, 39], [16, 17, 18, 26], [24, 25, 32, 33, 34], [0, 1, 8, 9, 10]]


In [36]:
for i in range(63):
  print(dat['Brodmann_Area'][i]=='Brodmann area 47')

True
True
False
False
False
False
False
False
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [26]:
'Brodmann Area '+str(47)

'Brodmann Area 47'

In [14]:
i=0
j=1
dat['Brodmann_Area'][i]==['Brodmann Area '+str(j)]

False

In [56]:
np.unique(dat['lobe'])

array(['Frontal Lobe', 'Parietal Lobe', 'Temporal Lobe'], dtype='<U13')

BA 6-10, 32,40,45-47

In [9]:
BA_list=['6','10','32','40','45','46','47']

In [8]:
dat['Brodmann_Area']

['Brodmann area 47',
 'Brodmann area 47',
 'Brodmann area 38',
 'Brodmann area 38',
 'Brodmann area 21',
 'Brodmann area 21',
 'Brodmann area 21',
 'Brodmann area 21',
 'Brodmann area 47',
 'Brodmann area 47',
 'Brodmann area 47',
 'Brodmann area 22',
 'Brodmann area 22',
 'Brodmann area 22',
 'Brodmann area 22',
 'Brodmann area 22',
 'Brodmann area 45',
 'Brodmann area 45',
 'Brodmann area 45',
 'Brodmann area 44',
 'Brodmann area 6',
 'Brodmann area 43',
 'Brodmann area 40',
 'Brodmann area 42',
 'Brodmann area 46',
 'Brodmann area 46',
 'Brodmann area 45',
 'Brodmann area 44',
 'Brodmann area 6',
 'Brodmann area 4',
 'Brodmann area 43',
 'Brodmann area 40',
 'Brodmann area 46',
 'Brodmann area 46',
 'Brodmann area 46',
 'Brodmann area 9',
 'Brodmann area 6',
 'Brodmann area 6',
 'Brodmann area 3',
 'Brodmann area 40',
 'Brodmann area 9',
 'Brodmann area 9',
 'Brodmann area 9',
 'Brodmann area 8',
 'Brodmann area 6',
 'Brodmann area 6',
 'Brodmann area 3',
 'Brodmann area 2',
 'Brodm

In [ ]:
store=[]